# <center> Université Paris Saclay M2 Data Science </center>#
# <center> TEXT MINING CHATBOTS </center>#
# <center> Named Entity Recognition assignement </center>#

### Names : Salma Damak
### Names :  Haytham AIT ETTAJER
### Names: Panongbene Jean Mohammed SAWADOGO 

In [ ]:
import re
import string
! pip install fasttext 
import fasttext
import unicodedata 
from time import time
from nltk.corpus import stopwords
from gensim.models import FastText
from gensim.models import  Word2Vec

     |████████████████████████████████| 71kB 3.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3093878 sha256=7e8949d11da290e091ecb75695dde2c21a3a79150c90dc194cf77639d62d8833
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
# French Stop Word
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Extract file directly in my drive
#driveAcces = '! 4/1AY0e-g5d6M3fRQ8bWMf_2L5f47GkimSXpf5o1pgZaDvBg_0tcenbPNYr6Q0'
#https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
#from google.colab import drive
#drive.mount("/content/drive/", force_remount=True)

#!ls drive/MyDrive/'Colab Notebooks'/data

/bin/bash: Extract: command not found
/bin/bash: driveAcces: command not found
from: can't read /var/mail/google.colab
/bin/bash: -c: line 0: syntax error near unexpected token `"/content/drive/",'
/bin/bash: -c: line 0: `drive.mount("/content/drive/", force_remount=True)'
ls: cannot access 'drive/MyDrive/Colab Notebooks': No such file or directory


In [ ]:
# file upload data
with open('drive/MyDrive/ColabNotebooks/Data/QUAERO_FrenchPress_traindev.ospl', 'r') as f: 
        QUAERO_FrenchPress = f.readlines()
        
# file upload data
with open('drive/MyDrive/ColabNotebooks/Data/QUAERO_FrenchMed_traindev.ospl', 'r') as f: 
        QUAERO_FrenchMed = f.readlines()


In [ ]:
# clean word
def clean_word(data):
    """This function remove all word and caracteres that we dont need"""
    return ''.join(x for x in unicodedata.normalize('NFKD', data) if x in string.ascii_letters).lower() 

In [ ]:
def build_token_word(document):
    """This function take a document and build a list of words"""
    list_values = list()
    for lines in document:
    #words=lines.split(' ')
        sent = re.sub("[^A-Za-z]+", ' ', str(lines))
        words_value=sent.split()
        list_values.append(words_value)
        
    return list_values

In [ ]:
# build model vocabulary
#### Building CBOW wordvectors
def training_cbow(document):
    """Training the cbow model"""
    t = time()
    # initialize
    cbow_model = Word2Vec(min_count=1,window=2,size=100, sg = 0)
    # build model vocabulary

    tokens_list=document
  
    cbow_model.build_vocab(tokens_list)
    # train the model
    cbow_model.train(tokens_list, total_examples=cbow_model.corpus_count, epochs=30, report_delay=1)

    print('Time to build CBOW model vocab: {} mins'.format(round((time() - t) / 60, 2)))
    return cbow_model

In [ ]:
def training_sg(document):
    """Train the skip gram model"""
    t = time()
    # initialize skipgram model
    sg_model = Word2Vec(min_count=1,window=2,size=100, sg = 1,sample=5e-5, alpha=0.05, min_alpha=0.0005 )
    # build model vocabulary
    sg_model.build_vocab(document)

    # train the model
    sg_model.train(document, total_examples=sg_model.corpus_count, epochs=30, report_delay=1)

    print('Time to build Skip gram model vocab: {} mins'.format(round((time() - t) / 60, 2)))
    return sg_model
  

In [ ]:
def stop_word(document):
    """This function take a list of list of word and delete stop word"""
    
    for i in range(len(document)):
        for j in document[i]:
            if (j in stop_words) or (len(j)<=2):
                document[i].remove(j)
    
    return document

In [ ]:
# build tocken
QUAERO_FrenchPress_tocken = stop_word(build_token_word(QUAERO_FrenchPress))
QUAERO_FrenchMed_tocken = stop_word(build_token_word(QUAERO_FrenchMed))

In [ ]:
# Word 2 Vec model
model = Word2Vec(min_count=1,size=100)
model = Word2Vec(min_count=1)
model.build_vocab(QUAERO_FrenchPress_tocken) # prepare the model vocabulary
model.train(QUAERO_FrenchPress_tocken, total_examples=model.corpus_count, epochs=3)#report_delay=1) 

(2095580, 2320746)

In [ ]:
model.wv.most_similar(["bonjour"])

[('bienvenu', 0.957136869430542),
 ('Winckler', 0.9519490003585815),
 ('messieurs', 0.9517946243286133),
 ('bienvenue', 0.9492441415786743),
 ('mesdames', 0.9420086145401001),
 ('Baudelot', 0.9373589158058167),
 ('Rocard', 0.9367847442626953),
 ('tentatrices', 0.9332044124603271),
 ('Rincquesen', 0.930302083492279),
 ('Lecaplain', 0.9293521642684937)]

In [ ]:
# Cbow model
cbow_QUAERO_FrenchPress_tocken = training_cbow(QUAERO_FrenchPress_tocken)
cbow_QUAERO_FrenchMed_tocken = training_cbow(QUAERO_FrenchMed_tocken)

Time to build CBOW model vocab: 0.78 mins
Time to build CBOW model vocab: 0.05 mins


In [ ]:
#  skipgram model
sg_QUAERO_FrenchPress_tocken = training_sg(QUAERO_FrenchPress_tocken)
sg_QUAERO_FrenchMed_tocken = training_sg(QUAERO_FrenchMed_tocken)

Time to build Skip gram model vocab: 1.06 mins
Time to build Skip gram model vocab: 0.08 mins


In [ ]:
# test skip gram model
sg_QUAERO_FrenchPress_tocken.wv.most_similar(["traitement"])

[('geait', 0.6342461109161377),
 ('dicamenteux', 0.6309572458267212),
 ('antidouleur', 0.6250557899475098),
 ('Asilah', 0.5593647956848145),
 ('suivait', 0.5587955117225647),
 ('paludisme', 0.5259395837783813),
 ('admirait', 0.5251320600509644),
 ('sahraoui', 0.5154713988304138),
 ('psychiatrique', 0.5103509426116943),
 ('galitaire', 0.5088360905647278)]

In [ ]:
sg_QUAERO_FrenchMed_tocken.wv.most_similar(["traitement"])

[('arr', 0.9944697618484497),
 ('Tasmar', 0.9941691160202026),
 ('TYSABRI', 0.9937978982925415),
 ('utilis', 0.9923129081726074),
 ('est', 0.9921622276306152),
 ('inform', 0.9919185638427734),
 ('par', 0.9918251037597656),
 ('pendant', 0.9917889833450317),
 ('instaur', 0.9917441010475159),
 ('une', 0.9915977120399475)]

In [ ]:
sg_QUAERO_FrenchPress_tocken.wv.most_similar(["patient"])

[('infantilis', 0.7661058902740479),
 ('neurologique', 0.6504168510437012),
 ('virale', 0.6334140300750732),
 ('humble', 0.6283608078956604),
 ('ventif', 0.6152496337890625),
 ('Tourcoing', 0.6135792136192322),
 ('contagieux', 0.6117741465568542),
 ('signaient', 0.6034929752349854),
 ('illogique', 0.6027514934539795),
 ('transmissible', 0.6003401875495911)]

In [ ]:
sg_QUAERO_FrenchMed_tocken.wv.most_similar(["patient"])

[('peuvent', 0.9967723488807678),
 ('lors', 0.9966742396354675),
 ('s', 0.9965825080871582),
 ('ces', 0.9965600371360779),
 ('survenir', 0.9964410066604614),
 ('tout', 0.9963569045066833),
 ('galement', 0.9962690472602844),
 ('cette', 0.9962559938430786),
 ('e', 0.9961977601051331),
 ('sympt', 0.9960789680480957)]

In [ ]:
sg_QUAERO_FrenchPress_tocken.wv.most_similar(["jaune"])

[('grenadine', 0.7031934261322021),
 ('caleront', 0.6713384389877319),
 ('maillot', 0.6655776500701904),
 ('Pena', 0.6150904893875122),
 ('Rock', 0.5896638631820679),
 ('USMA', 0.588503897190094),
 ('endossant', 0.5833740234375),
 ('franchie', 0.5803539752960205),
 ('Trissotin', 0.5779874324798584),
 ('Avenue', 0.5762203335762024)]

In [ ]:
sg_QUAERO_FrenchMed_tocken.wv.most_similar(["jaune"])

[('orange', 0.9974260330200195),
 ('Oxyde', 0.9972546100616455),
 ('anormale', 0.997245728969574),
 ('Ethylcellulose', 0.9971926808357239),
 ('thylcellulose', 0.9971528053283691),
 ('talc', 0.9970520734786987),
 ('oxyde', 0.9970391392707825),
 ('yeux', 0.9970212578773499),
 ('urines', 0.9969363808631897),
 ('hexagonaux', 0.9968926906585693)]

In [ ]:
dimension =100
fasttext_QUAERO_FrenchMed_tocken = FastText(QUAERO_FrenchMed_tocken, size=dimension, window=2, min_count=1, workers=4, sg=0)
fasttext_QUAERO_FrenchPress_tocken = FastText(QUAERO_FrenchPress_tocken, size=dimension, window=2, min_count=1, workers=4, sg=0)

In [ ]:
fasttext_QUAERO_FrenchMed_tocken.wv.most_similar(["traitement"])

[('traitements', 0.9999978542327881),
 ('Traitement', 0.9999977350234985),
 ('Taaitement', 0.9999971389770508),
 ('allaitement', 0.999996542930603),
 ('Allaitement', 0.9999961256980896),
 ('vitement', 0.9999955892562866),
 ('Traitements', 0.9999954104423523),
 ('troitement', 0.999995231628418),
 ('lentement', 0.9999949932098389),
 ('partement', 0.9999944567680359)]

In [ ]:
fasttext_QUAERO_FrenchPress_tocken.wv.most_similar(["traitement"])

[('vachement', 0.9852569103240967),
 ('chement', 0.984168291091919),
 ('troitement', 0.982939600944519),
 ('sauvagement', 0.9826259016990662),
 ('chauffement', 0.9823418259620667),
 ('chichement', 0.9819760322570801),
 ('farouchement', 0.9816651344299316),
 ('sagement', 0.9814537763595581),
 ('changement', 0.9812566041946411),
 ('doucement', 0.9809319376945496)]

In [ ]:
fasttext_QUAERO_FrenchMed_tocken.wv.most_similar(["jaune"])

[('antithrombotiques', 0.999937891960144),
 ('sanctions', 0.999936580657959),
 ('consommation', 0.9999365210533142),
 ('instrumentale', 0.9999363422393799),
 ('mentionn', 0.9999362230300903),
 ('antithrombotique', 0.9999353885650635),
 ('bronchographiques', 0.9999344944953918),
 ('Malformation', 0.9999344348907471),
 ('concentration', 0.9999344348907471),
 ('niques', 0.9999344348907471)]

In [ ]:
fasttext_QUAERO_FrenchPress_tocken.wv.most_similar(["jaune"])

[('Neptune', 0.990755558013916),
 ('lune', 0.9892677664756775),
 ('brune', 0.9860022664070129),
 ('Jeune', 0.9837058782577515),
 ('dune', 0.9827749133110046),
 ('youne', 0.982375979423523),
 ('lagune', 0.9764001369476318),
 ('aune', 0.971815824508667),
 ('doune', 0.9616575241088867),
 ('Tribune', 0.9520894289016724)]

In [ ]:
fasttext_QUAERO_FrenchMed_tocken.wv.most_similar(["Picasso"])

[('iso', 0.9651605486869812),
 ('guid', 0.9626452922821045),
 ('ANASTOMOSES', 0.9624209403991699),
 ('Typho', 0.9619781970977783),
 ('isol', 0.9619286060333252),
 ('Emil', 0.9616976976394653),
 ('THYROUIDE', 0.9615924954414368),
 ('isolee', 0.9615694284439087),
 ('Cas', 0.9614438414573669),
 ('Bignami', 0.9613182544708252)]

In [ ]:
fasttext_QUAERO_FrenchPress_tocken.wv.most_similar(["Picasso"])

[('Ricardo', 0.9692336320877075),
 ('Bangladesh', 0.964695394039154),
 ('Stangl', 0.9602813720703125),
 ('Dassault', 0.9576665163040161),
 ('Golinsky', 0.9569246768951416),
 ('Fasso', 0.9564516544342041),
 ('Bricorama', 0.9562460780143738),
 ('Mitrovica', 0.9561678767204285),
 ('Ouganda', 0.9555414915084839),
 ('Rosinski', 0.9552026987075806)]

# Named entity recognition



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/XuezheMax/NeuroNLP2.git
!pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install overrides
!pip uninstall scipy
!pip --no-cache-dir install scipy==1.1
!cd NeuroNLP2
#!pip install .

fatal: destination path 'NeuroNLP2' already exists and is not an empty directory.
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Uninstalling scipy-1.1.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.1.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/uninstall.py", line 79, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_install.py", line 755, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_uninstall.py", line 388, in remove
    if auto_confirm or self._allowed_to_pro

In [ ]:
#! git clone https://github.com/allenai/spv2.git 
! cd NeuroNLP2/experiments & mkdir data
! mkdir NeuroNLP2/experiments/data
! mkdir NeuroNLP2/experiments/data/Press
! mkdir NeuroNLP2/experiments/data/Med
! mkdir NeuroNLP2/experiments/data/glove 
! mkdir NeuroNLP2/experiments/data/glove/glove.6B
! cp drive/MyDrive/ColabNotebooks/Data/quaero-100-demi.json NeuroNLP2/experiments/data/Press/
! cp drive/MyDrive/ColabNotebooks/Data/QUAERO_FrenchPress-w2v.vec.gz NeuroNLP2/experiments/data/Press/
! cp drive/MyDrive/ColabNotebooks/Data/glove.6B.100d.gz NeuroNLP2/experiments/data/glove/glove.6B/
! cp drive/MyDrive/ColabNotebooks/Data/fra4_ID.train NeuroNLP2/experiments/data/Press/
! cp drive/MyDrive/ColabNotebooks/Data/fra4_ID.dev NeuroNLP2/experiments/data/Press/
! cp drive/MyDrive/ColabNotebooks/Data/fra4_ID.test NeuroNLP2/experiments/data/Press/
! cp drive/MyDrive/ColabNotebooks/Data/QUAERO_FrenchPress_traindev.ospl NeuroNLP2/experiments/data/Press/
! cp drive/MyDrive/ColabNotebooks/Data/QUAERO_FrenchMed_traindev.ospl  NeuroNLP2/experiments/data/Med/
! cp drive/MyDrive/ColabNotebooks/Data/EMEA/EMEAdev_layer1_ID.conll NeuroNLP2/experiments/data/Med/
! cp drive/MyDrive/ColabNotebooks/Data/EMEA/EMEAtest_layer1_ID.conll NeuroNLP2/experiments/data/Med/
! cp drive/MyDrive/ColabNotebooks/Data/EMEA/EMEAtrain_layer1_ID.conll NeuroNLP2/experiments/data/Med/
! cp drive/MyDrive/ColabNotebooks/Data/MEDLINE/MEDLINEdev_layer1_ID.conll NeuroNLP2/experiments/data/Med/
! cp drive/MyDrive/ColabNotebooks/Data/MEDLINE/MEDLINEtest_layer1_ID.conll NeuroNLP2/experiments/data/Med/
! cp drive/MyDrive/ColabNotebooks/Data/MEDLINE/MEDLINEtrain_layer1_ID.conll NeuroNLP2/experiments/data/Med/
#! rm NeuroNLP2/experiments/scripts/run_ner_conll03.sh
#! cp drive/MyDrive/ColabNotebooks/Data/run_ner_conll03.sh NeuroNLP2/experiments/scripts/
#! mv NeuroNLP2/experiments/data/glove/glove.6B/glove.6B.100d.txt.gz NeuroNLP2/experiments/data/glove/glove.6B/glove.6B.100d.gz


In [ ]:
##--------------------------------------FICHIER SCRIPT-----------------------------------
#!/usr/bin/env bash
CUDA_VISIBLE_DEVICES=0 OMP_NUM_THREADS=4 python NeuroNLP2/experiments/ner.py --config NeuroNLP2/experiments/configs/ner/conll03.json --num_epochs 20 --batch_size 16 \
 --loss_type sentence --optim sgd --learning_rate 0.01 --lr_decay 0.99999 --grad_clip 0.0 --warmup_steps 100 --weight_decay 0.0 --unk_replace 0.0 \
 --embedding glove --embedding_dict "NeuroNLP2/experiments/data/glove/glove.6B/glove.6B.100d.gz" --model_path "models/ner/conll03" \
 --train "NeuroNLP2/experiments/data/fra4_ID.train" --dev "NeuroNLP2/experiments/data/fra4_ID.dev" --test "NeuroNLP2/experiments/data/fra4_ID.test"


In [ ]:
##--------------------------------------FICHIER SCRIPT BIS-----------------------------------
#!/usr/bin/env bash
CUDA_VISIBLE_DEVICES=0 OMP_NUM_THREADS=60 python NeuroNLP2/experiments/ner.py --config NeuroNLP2/experiments/data/Press/quaero-100-demi.json --num_epochs 20 --batch_size 16 \
 --loss_type sentence --optim sgd --learning_rate 0.01 --lr_decay 0.99999 --grad_clip 0.0 --warmup_steps 10 --weight_decay 0.0 --unk_replace 0.0 \
 --embedding sskip --embedding_dict "NeuroNLP2/experiments/data/Press/QUAERO_FrenchPress-w2v.vec.gz" --model_path "models/ner/ner-fra4_ID-w2v-conll17-half" \
 --train "NeuroNLP2/experiments/data/Press/fra4_ID.train" --dev "NeuroNLP2/experiments/data/Press/fra4_ID.dev" --test "NeuroNLP2/experiments/data/Press/fra4_ID.test"

In [ ]:
##--------------------------------------FICHIER SCRIPT MEDLINEGLOVE-----------------------------------
#!/usr/bin/env bash
CUDA_VISIBLE_DEVICES=0 OMP_NUM_THREADS=60 python NeuroNLP2/experiments/ner.py --config NeuroNLP2/experiments/data/Press/quaero-100-demi.json --num_epochs 20 --batch_size 16 \
 --loss_type sentence --optim sgd --learning_rate 0.01 --lr_decay 0.99999 --grad_clip 0.0 --warmup_steps 10 --weight_decay 0.0 --unk_replace 0.0 \
 --embedding sskip --embedding_dict "NeuroNLP2/experiments/data/glove/glove.6B/glove.6B.100d.gz" --model_path "models/ner/ner-fra4_ID-w2v-conll17-half" \
 --train "NeuroNLP2/experiments/data/Med/MEDLINEtrain_layer1_ID.conll" --dev "NeuroNLP2/experiments/data/Med/MEDLINEdev_layer1_ID.conll" --test "NeuroNLP2/experiments/data/Med/MEDLINEtest_layer1_ID.conll"

In [ ]:
##--------------------------------------FICHIER SCRIPT MEDLINEGLOVE-----------------------------------
#!/usr/bin/env bash
CUDA_VISIBLE_DEVICES=0 OMP_NUM_THREADS=60 python NeuroNLP2/experiments/ner.py --config NeuroNLP2/experiments/data/Press/quaero-100-demi.json --num_epochs 20 --batch_size 16 \
 --loss_type sentence --optim sgd --learning_rate 0.01 --lr_decay 0.99999 --grad_clip 0.0 --warmup_steps 10 --weight_decay 0.0 --unk_replace 0.0 \
 --embedding sskip --embedding_dict "drive/MyDrive/ColabNotebooks/Data/Text Mining/models Press/ner/ner-fra4_ID-w2v-conll17-half/model.pt" --model_path "models/ner/ner-fra4_ID-w2v-conll17-half" \
 --train "NeuroNLP2/experiments/data/fra4_ID.train" --dev "NeuroNLP2/experiments/data/fra4_ID.dev" --test "NeuroNLP2/experiments/data/fra4_ID.test"

In [ ]:
! ./NeuroNLP2/experiments/scripts/run_ner_conll03.sh

Namespace(amsgrad=False, batch_size=16, config='NeuroNLP2/experiments/data/Press/quaero-100-demi.json', cuda=False, dev='NeuroNLP2/experiments/data/fra4_ID.dev', embedding='sskip', embedding_dict='drive/MyDrive/ColabNotebooks/Data/Text Mining/models Press/ner/ner-fra4_ID-w2v-conll17-half/model.pt', grad_clip=0.0, learning_rate=0.01, loss_type='sentence', lr_decay=0.99999, model_path='models/ner/ner-fra4_ID-w2v-conll17-half', num_epochs=20, optim='sgd', test='NeuroNLP2/experiments/data/fra4_ID.test', train='NeuroNLP2/experiments/data/fra4_ID.train', unk_replace=0.0, warmup_steps=10, weight_decay=0.0)
loading embedding: sskip from drive/MyDrive/ColabNotebooks/Data/Text Mining/models Press/ner/ner-fra4_ID-w2v-conll17-half/model.pt
Traceback (most recent call last):
  File "NeuroNLP2/experiments/ner.py", line 316, in <module>
    main()
  File "NeuroNLP2/experiments/ner.py", line 121, in main
    embedd_dict, embedd_dim = utils.load_embedding_dict(embedding, embedding_path)
  File "/conten

In [ ]:
! ./NeuroNLP2/experiments/scripts/run_neuromst.sh

Namespace(amsgrad=False, batch_size=16, config='NeuroNLP2/experiments/data/Press/quaero-100-demi.json', cuda=False, dev='NeuroNLP2/experiments/data/Med/EMEAdev_layer1_ID.conll', embedding='sskip', embedding_dict='NeuroNLP2/experiments/data/glove/glove.6B/glove.6B.100d.gz', grad_clip=0.0, learning_rate=0.01, loss_type='sentence', lr_decay=0.99999, model_path='models/ner/EMEA-Glove', num_epochs=20, optim='sgd', test='NeuroNLP2/experiments/data/Med/EMEAtest_layer1_ID.conll', train='NeuroNLP2/experiments/data/Med/EMEAtrain_layer1_ID.conll', unk_replace=0.0, warmup_steps=10, weight_decay=0.0)
loading embedding: sskip from NeuroNLP2/experiments/data/glove/glove.6B/glove.6B.100d.gz
2021-02-11 16:31:08,747 - NER - INFO - Creating Alphabets
2021-02-11 16:31:08,747 - Create Alphabets - INFO - Creating Alphabets: models/ner/EMEA-Glove/alphabets
2021-02-11 16:31:08,792 - Create Alphabets - INFO - Total Vocabulary Size: 2465
2021-02-11 16:31:08,793 - Create Alphabets - INFO - Total Singleton Size: 

In [ ]:
! ./NeuroNLP2/experiments/scripts/run_deepbiaf.sh

Namespace(amsgrad=False, batch_size=16, config='NeuroNLP2/experiments/data/Press/quaero-100-demi.json', cuda=False, dev='NeuroNLP2/experiments/data/Med/MEDLINEdev_layer1_ID.conll', embedding='sskip', embedding_dict='NeuroNLP2/experiments/data/Press/QUAERO_FrenchPress-w2v.vec.gz', grad_clip=0.0, learning_rate=0.01, loss_type='sentence', lr_decay=0.99999, model_path='models/ner/Medline-Press', num_epochs=20, optim='sgd', test='NeuroNLP2/experiments/data/Med/MEDLINEtest_layer1_ID.conll', train='NeuroNLP2/experiments/data/Med/MEDLINEtrain_layer1_ID.conll', unk_replace=0.0, warmup_steps=10, weight_decay=0.0)
loading embedding: sskip from NeuroNLP2/experiments/data/Press/QUAERO_FrenchPress-w2v.vec.gz
2021-02-11 16:27:39,420 - NER - INFO - Creating Alphabets
2021-02-11 16:27:39,420 - Create Alphabets - INFO - Creating Alphabets: models/ner/Medline-Press/alphabets
2021-02-11 16:27:39,450 - Create Alphabets - INFO - Total Vocabulary Size: 3783
2021-02-11 16:27:39,451 - Create Alphabets - INFO -

In [ ]:
! ./NeuroNLP2/experiments/scripts/run_analyze.sh

Namespace(amsgrad=False, batch_size=16, config='NeuroNLP2/experiments/data/Press/quaero-100-demi.json', cuda=False, dev='NeuroNLP2/experiments/data/Med/EMEAdev_layer1_ID.conll', embedding='sskip', embedding_dict='NeuroNLP2/experiments/data/Press/QUAERO_FrenchPress-w2v.vec.gz', grad_clip=0.0, learning_rate=0.01, loss_type='sentence', lr_decay=0.99999, model_path='models/ner/EMEA-Press', num_epochs=20, optim='sgd', test='NeuroNLP2/experiments/data/Med/EMEAtest_layer1_ID.conll', train='NeuroNLP2/experiments/data/Med/EMEAtrain_layer1_ID.conll', unk_replace=0.0, warmup_steps=10, weight_decay=0.0)
loading embedding: sskip from NeuroNLP2/experiments/data/Press/QUAERO_FrenchPress-w2v.vec.gz
2021-02-11 16:33:48,115 - NER - INFO - Creating Alphabets
2021-02-11 16:33:48,115 - Create Alphabets - INFO - Creating Alphabets: models/ner/EMEA-Press/alphabets
2021-02-11 16:33:48,163 - Create Alphabets - INFO - Total Vocabulary Size: 2465
2021-02-11 16:33:48,163 - Create Alphabets - INFO - Total Singleto

In [ ]:
! cp /content/models/ drive/MyDrive/ColabNotebooks/Data/MEDLINE/

cp: -r not specified; omitting directory '/content/models/'


In [ ]:
#import torch
#model = torch.load("drive/MyDrive/ColabNotebooks/Data/Text Mining/models Press/ner/ner-fra4_ID-w2v-conll17-half/model.pt")
#print(model)
#model.values.

AttributeError: ignored